In [105]:
# load libraries
from sklearn.cluster import AgglomerativeClustering, KMeans
from nilearn import input_data
from numpy import save

import pandas as pd
import numpy as np
import pickle
import os

In [78]:
# load function

def get_dot(seed_time_series, region_time_series):
    
    '''function that takes as input the time series of a seed and time series of a region,
    and returns the correlation between the them
    '''    
    
    flattened_seed = np.ravel(seed_time_series)
    flattened_region = np.ravel(region_time_series)
    
    dot_p = np.dot(flattened_region,flattened_seed)/flattened_seed.shape[0]
    
    return dot_p

In [79]:
# load function

def time_series(coords, func, confounds):
    
    ''' Function that takes as input the seed coordinates, func & confounds files of a 
    subject and returns the time series of the seed
    '''
    mask = input_data.NiftiSpheresMasker(
    coords, radius=10,
    detrend=True, standardize=True,
    low_pass=0.1, high_pass=0.01, t_r=2,
    memory='nilearn_cache', memory_level=1, verbose=0)
    
    time_series = mask.fit_transform(func,
                confounds=[confounds])
    
    return time_series

In [80]:
# set path to files

ADNI_dir = '/Users/natashaclarke/Documents/brainhack/ADNI_matched'

In [81]:
# create a .txt file listing all confound files in your directory. We then loop through them and add them to a list

confound_data = []
conf_file = open((os.path.join(ADNI_dir, 'confound_files.txt')),'r')
for line in conf_file:
    confound_data.append(os.path.join(ADNI_dir,(line.strip())))
conf_file.close()

In [82]:
# create a .txt file listing all func files in your directory. We then loop through them and add them to a list

func_data = []
func_file = open((os.path.join(ADNI_dir, 'func_files.txt')),'r')
for line in func_file:
    func_data.append(os.path.join(ADNI_dir,(line.strip())))
func_file.close()

In [84]:
# check that there are as many files as you expected (and that the lengths match!)

print ('Length of func_data =', len(func_data))
print ('Length of confound_data =', len(confound_data))

Length of func_data = 20
Length of confound_data = 20


In [85]:
# sort the lists so they are in order

func_sorted = sorted(func_data)
confound_sorted = sorted(confound_data)

In [86]:
# set the seed coordinates. Current seed = left anterior medial temporal gyrus

seed_coords = [(-60, -6, 18)] 

In [87]:
# give coorrdinates of ROIs that we want to calculate connectivity with the seed

L_angular = [(-46,-62,34)]
medial_prefront = [(0,60,-8)]
post_cing = [(-3,-49,33)]
R_aMTG = [(60,-10,-18)]

In [88]:
# create a dictionary for the ROIs

regions_dict = {}

In [91]:
# add ROIs to dictionary with name as the key and coordinates as the value

regions_dict['L_angular'] = L_angular
regions_dict['medial_prefront'] = medial_prefront
regions_dict['post_cing'] = post_cing
regions_dict['R_aMTG'] = R_aMTG

In [92]:
'''Loop through the files in your sorted lists. For each, we first calculate the seed time series
and times series of each of our ROIs. Then, calculate the correlation between
the seed time series and each ROI time series and add them to a df'''

# create an empty df with the column headers of our ROIs
df_corr = pd.DataFrame(data=0, columns=regions_dict, index=range(0))

for func, confounds in zip(func_sorted, confound_sorted):
    subject_seed = time_series(seed_coords, func, confounds)  # calculate subject seed time-series

    region_series_list = []
    for region, coord in regions_dict.items():
        region_series = time_series(coord, func, confounds)   # calculate time series of each ROI
        region_series_list.append(region_series)
        
    correlation = []
    for series in region_series_list:
        dot = series_corr.get_dot(subject_seed, series)       # correlate seed and ROI time series
        correlation.append(dot)
    
    correlation = pd.Series(correlation, index = df_corr.columns)
    df_corr = df_corr.append(correlation, ignore_index=True)

In [93]:
# convert df to array for clustering

X = df_corr.to_numpy()

In [107]:
# save X to npy file for later

save('clustering_X.npy', X)

In [94]:
# add a column to the correlations df that lists the func file, and save to excel

df_corr['file'] = func_sorted
df_corr.to_excel('correlations.xlsx')

In [99]:
# initiate clustering models and fit to data. You can add/ change the models using the scikit learn documentation
# for the 2 Agglomerative clustering models, either set the distance_threshold or n_clusters

model1_ward = AgglomerativeClustering(linkage='ward', distance_threshold=None, n_clusters=2).fit(X)
model2_average = AgglomerativeClustering(linkage='average', distance_threshold=None, n_clusters=2).fit(X)
model3_KMeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [100]:
# pickle models to save to disk

pickle.dump(model1_ward, open('model1_ward.sav', 'wb'))
pickle.dump(model2_average, open('model2_average.sav', 'wb'))
pickle.dump(model3_KMeans, open('model3_KMeans.sav', 'wb'))

In [103]:
# Congratulations! You just trained three clustering models using seed-based functional connectivity!
# Head over to the clustering_visualization notebook to explore how your clustering algorithms did